In [1]:
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 22})
import numpy as np
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
from parse_cube_files import CUBE

# dsgdb9nsd_002626

In [2]:
ve38 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_38.cube')
ve30 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_30.cube')
ve23 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_23.cube')
ve15 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_15.cube')
ve8 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/dsgdb9nsd_002626/DENS_8.cube')
ve0 = CUBE(r'/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/lambda_integrals/free_electron_gas.cube')

# Integration

In [9]:
from alchemy_tools import integrate_lambda_density, get_atomic_energies

In [4]:
lambda_densities = [ve0, ve8, ve15, ve23, ve30, ve38]
density_arrays = []
for dens in lambda_densities:
    dens.scale()
    density_arrays.append(dens.data_scaled)
lam_vals = [0, 8/38, 15/38, 23/38, 30/38, 1]

In [5]:
av_dens = integrate_lambda_density(density_arrays, lam_vals)

# Plots

In [6]:
# projection on line
x = np.linspace(0, 20, 175)
sa = (1,2)
av_dens_1D = av_dens.sum(axis=sa)
pd_1D = [ve38.project(sa), ve30.project(sa), ve23.project(sa), ve15.project(sa), ve8.project(sa), ve0.project(sa), av_dens_1D]
label = [r'$\rho(\lambda = 1.0)$',r'$\rho(\lambda \approx 0.79)$', r'$\rho(\lambda \approx 0.61)$', r'$\rho(\lambda \approx 0.39)$', r'$\rho(\lambda \approx 0.21)$', r'$\rho(\lambda = 0)$', r'$\tilde{\rho} \approx \int_0^1 d\lambda \rho(\lambda, r)$']
fig, ax = plt.subplots(1,1)
for idx, pd in enumerate(pd_1D):
    # bigger linewidth for lambda-averaged density
    if idx == len(pd_1D)-1:
        ax.plot(x, pd, '--', label=label[idx])
    else:
        ax.plot(x, pd, label=label[idx])

ax.set_title('Boxsize = 20 Ang')
ax.set_xlabel('Cell coordinate (Ang)')
ax.set_xlim(0.5, 19.5)
ax.set_ylabel(r'Projected Density $\rho(x_0)$ (Bohr$^{-3}$) ')

ax.legend()

# Alchemical Potential and Atomic Energies

In [7]:
nuclei = ve38.atoms
meshgrid = ve38.get_grid()

In [10]:
atomic_energies = get_atomic_energies(av_dens, nuclei, meshgrid)

In [11]:
atomic_energies

[52.09381063770665,
 38.296932567509344,
 39.413724788931646,
 39.536336953667266,
 38.60430754778666,
 36.191146203702715,
 32.34069350226706,
 5.24609477753838,
 5.788707564857359,
 5.625141615251788,
 5.632831881484248,
 5.759055176641796,
 5.644759681641726,
 5.519526689025201,
 4.5011333982786885]

In [13]:
nuclei[0][1]


8

In [14]:
dist_gpt_nuc = np.sqrt(np.power(meshgrid[0]-nuclei[0][1], 2) + np.power(meshgrid[1]-nuclei[0][2], 2) + np.power(meshgrid[2]-nuclei[0][3], 2))

In [23]:
np.sum(av_dens/dist_gpt_nuc)

6.511726329713332